In [1]:
#Import packages

#Rdkit
from rdkit import Chem

#Scipy
from scipy import stats 
from scipy.stats import uniform,ranksums 

#Pandas
import pandas as pd
from pandas import DataFrame

#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib #jbolib模块
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [2]:
import torch
import deepchem as dc
import numpy as np

In [3]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [4]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\CAL-27")

In [22]:
df = pd.read_csv('./pubchem_cellid_3570_bioactivity.csv',index_col=0)
cid = pd.read_csv('./data.csv',index_col=0)

In [23]:
df.index=df['cid']
df['IsomericSMILES']=cid['IsomericSMILES']

In [24]:
df

,acvalue,aid,sid,cid,geneid,pmid,aidtype,aidmdate,hasdrc,rnai,...,targetname,targeturl,ecs,repacxn,taxid,cellids,targettaxid,anatomyid,anatomy,IsomericSMILES
cid,,,,,,,,,,,,,,,,,,,,,
446378,0.000169,742002,104234196,446378,NaN,NaN,Confirmatory,20220830,0,0,...,NaN,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,CCCCCCCC(=O)O[C@H]1[C@H]2C(=C([C@@H]1OC(=O)/C(...
148123,0.000339,742002,312468102,148123,NaN,NaN,Confirmatory,20220830,0,0,...,NaN,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@]([C@...
448013,0.001387,742002,103306054,448013,NaN,NaN,Confirmatory,20220830,0,0,...,NaN,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,C[C@H]1CCC[C@@]2([C@@H](O2)C[C@H](OC(=O)C[C@@H...
24360,0.002018,742002,103168388,24360,NaN,NaN,Confirmatory,20220830,0,0,...,NaN,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=...
13342,0.002086,742002,103229487,13342,NaN,NaN,Confirmatory,20220830,0,0,...,NaN,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,CC[C@@]1(C[C@H]2C[C@@](C3=C(CCN(C2)C1)C4=CC=CC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166633521,NaN,1823489,475994567,166633521,NaN,35450378.0,Literature-derived,20230314,0,0,...,NaN,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,CN1CC2=NN(C(=C2C3=CC4=C(NC(=C4C5=CN(CCCC1=O)N=...
24748573,NaN,1823489,104246358,24748573,NaN,35450378.0,Literature-derived,20230314,0,0,...,NaN,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,C1=CC(=CC(=C1)Cl)NC2=NC3=C(C=CC(=C3)C(=O)O)C4=...
166633407,NaN,1823489,475994345,166633407,NaN,35450378.0,Literature-derived,20230314,0,0,...,NaN,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,CN1CC2=NN(C(=C2C3=CC4=C(NC=C4C5=CN(CCCC1=O)N=C...


In [25]:
df.dropna(subset=["cid"], inplace=True)  
df['Activity']=np.where(df['activity']=='Active', 1, 0)
df.sort_values(by="Activity" , inplace=True, ascending=False) 
df.drop_duplicates(subset='cid', keep='first', inplace=True)
df=df.drop(['cid'], axis=1)
df

,acvalue,aid,sid,geneid,pmid,aidtype,aidmdate,hasdrc,rnai,activity,...,targeturl,ecs,repacxn,taxid,cellids,targettaxid,anatomyid,anatomy,IsomericSMILES,Activity
cid,,,,,,,,,,,,,,,,,,,,,
446378,0.000169,742002,104234196,NaN,NaN,Confirmatory,20220830,0,0,Active,...,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,CCCCCCCC(=O)O[C@H]1[C@H]2C(=C([C@@H]1OC(=O)/C(...,1
155543375,4.210000,1600733,440164015,NaN,31431326.0,Confirmatory,20220830,0,0,Active,...,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,C1=CC=C2C(=C1)C=CN=C2C(=O)NOCC3=CC=C(C=C3)/C=C...,1
155549392,5.250000,1600733,440179093,NaN,31431326.0,Confirmatory,20220830,0,0,Active,...,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,CC1=C(C2=CC=CC=C2N1)CC(=O)NOCC3=CC=C(C=C3)/C=C...,1
10033008,5.250000,1600856,103601300,NaN,31443950.0,Confirmatory,20220830,0,0,Active,...,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,C[C@@H]1CC(=O)C2=C(C3=C(C=CC(=C3O[C@@]2([C@@H]...,1
42625508,5.011870,414744,103611507,NaN,19301902.0,Confirmatory,20220830,0,0,Active,...,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,CC1=C(C=C(C=C1)NCC2=CC=C(C=C2)/C=C/C(=O)NO)NC3...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609586,35.191100,742002,164127604,NaN,NaN,Confirmatory,20220830,0,0,Unspecified,...,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,C1=CC=C2C(=C1)C(=CN2)CCNC3=CC=C(C=C3)NC4=CC=NC=C4,0
92577,35.827500,742002,103587081,NaN,NaN,Confirmatory,20220830,0,0,Unspecified,...,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,C1=CC2=C(C=C(C(=C2N=C1)O)N=NC3=CC4=C(C=C3)C=C(...,0
11433190,37.723100,742002,103459505,NaN,NaN,Confirmatory,20220830,0,0,Unspecified,...,NaN,NaN,NaN,NaN,3570,NaN,NaN,NaN,CC(C)OC1=C(C=CC(=C1)OC)C2=N[C@H]([C@H](N2C(=O)...,0


In [26]:

Original_data=pd.DataFrame(df['IsomericSMILES'])
Original_data['Activity']=df['Activity']
Chemdraw_smiles=Original_data['IsomericSMILES'].tolist()

Original_data.head()

,IsomericSMILES,Activity
cid,,
446378,CCCCCCCC(=O)O[C@H]1[C@H]2C(=C([C@@H]1OC(=O)/C(...,1
155543375,C1=CC=C2C(=C1)C=CN=C2C(=O)NOCC3=CC=C(C=C3)/C=C...,1
155549392,CC1=C(C2=CC=CC=C2N1)CC(=O)NOCC3=CC=C(C=C3)/C=C...,1
10033008,C[C@@H]1CC(=O)C2=C(C3=C(C=CC(=C3O[C@@]2([C@@H]...,1
42625508,CC1=C(C=C(C=C1)NCC2=CC=C(C=C2)/C=C/C(=O)NO)NC3...,1


In [27]:
Smis=[]
for i in Chemdraw_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Original_data['Canonical_smiles']=Canonical_smis
Original_data.to_csv("./Original_data.csv",sep=',',header=1,index=1)
Original_data

,IsomericSMILES,Activity,Canonical_smiles
cid,,,
446378,CCCCCCCC(=O)O[C@H]1[C@H]2C(=C([C@@H]1OC(=O)/C(...,1,C/C=C(/C)C(=O)O[C@H]1C(C)=C2[C@@H]3OC(=O)[C@@]...
155543375,C1=CC=C2C(=C1)C=CN=C2C(=O)NOCC3=CC=C(C=C3)/C=C...,1,O=C(/C=C/c1ccc(CONC(=O)c2nccc3ccccc23)cc1)NO
155549392,CC1=C(C2=CC=CC=C2N1)CC(=O)NOCC3=CC=C(C=C3)/C=C...,1,Cc1[nH]c2ccccc2c1CC(=O)NOCc1ccc(/C=C/C(=O)NO)cc1
10033008,C[C@@H]1CC(=O)C2=C(C3=C(C=CC(=C3O[C@@]2([C@@H]...,1,CC(=O)OC[C@]12Oc3c(-c4ccc(O)c5c4O[C@@]4(COC(C)...
42625508,CC1=C(C=C(C=C1)NCC2=CC=C(C=C2)/C=C/C(=O)NO)NC3...,1,Cc1ccc(NCc2ccc(/C=C/C(=O)NO)cc2)cc1Nc1nccc(-c2...
...,...,...,...
11609586,C1=CC=C2C(=C1)C(=CN2)CCNC3=CC=C(C=C3)NC4=CC=NC=C4,0,c1ccc2c(CCNc3ccc(Nc4ccncc4)cc3)c[nH]c2c1
92577,C1=CC2=C(C=C(C(=C2N=C1)O)N=NC3=CC4=C(C=C3)C=C(...,0,O=S(=O)(O)c1ccc2cc(N=Nc3cc(S(=O)(=O)O)c4cccnc4...
11433190,CC(C)OC1=C(C=CC(=C1)OC)C2=N[C@H]([C@H](N2C(=O)...,0,COc1ccc(C2=N[C@@H](c3ccc(Cl)cc3)[C@@H](c3ccc(C...


In [29]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [30]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis.tolist())
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_gelation=aDesc.get_output()

In [31]:
X_original_data=pd.DataFrame(Data_gelation)
Desc_names_names=aDesc.get_output_descriptors()
X_original_data.columns=Desc_names_names
X_original_data.index=Original_data.index

X_original_data.head()

,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,...,s1_numAroBonds,s2_numAroBonds,s3_numAroBonds,s4_numAroBonds,s34_size,s34_relSize,s34_phSize,s34_phRelSize,chiralMoment,chiralPhMoment
cid,,,,,,,,,,,,,,,,,,,,,
1401,523.76,6.629873,46.2498,78.7157,49.3472,90.1747,0.585441,0.996401,0.624648,1.141452,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
2375,430.41,10.009535,29.8367,45.7088,29.3183,49.4346,0.693877,1.062995,0.681821,1.149642,...,0.0,0.0,6.0,6.0,26.0,0.896552,5.0,0.172414,24.657656,4.472136
3218,294.43,6.264468,26.7076,46.7960,28.7162,53.2352,0.568247,0.995660,0.610983,1.132664,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
3385,130.09,10.840833,8.3825,13.2545,7.6193,14.1703,0.698542,1.104542,0.634942,1.180858,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
4091,129.20,6.460000,10.6874,20.1598,11.3127,23.7371,0.534370,1.007990,0.565635,1.186855,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000


In [32]:
#Remove descriptors with excessive NA values
Thresh =int(0.8*len(X_original_data))
X_NAomit_data=X_original_data.dropna(
    #thresh=Thresh, 
    axis=1) 
X_NAomit_data.head()
X_NAomit_data.to_csv("./X_NAomit_data.csv",sep=',',header=1,index=1)

In [33]:
#Ranksum test
Ranksums_data=X_NAomit_data
Ranksums_data["Activity"]=Original_data['Activity']
Notgelable=Ranksums_data.loc[Ranksums_data["Activity"] == 0]
Gelable=Ranksums_data.loc[Ranksums_data["Activity"] == 1]
Ranksums_result=[]
for i in list(Ranksums_data.columns):
 NG = Notgelable[i]
 G = Gelable[i]
 Ranksums_caculate=ranksums(NG, G, alternative='two-sided')
 Ranksums_result.append(Ranksums_caculate)
Ranksums_results = pd.DataFrame(Ranksums_result, columns=['Statistics','Pvalues'])
Ranksums_results.index=Ranksums_data.columns#
Ranksums_results = Ranksums_results.drop('Activity',axis = 0)
Ranksums_results.sort_values(by="Pvalues" , inplace=True, ascending=True)
Ranksums_results

,Statistics,Pvalues
SAdon,-5.661572,1.499923e-08
P_VSA_ppp_D,-5.659880,1.514787e-08
MATS3e,-5.389154,7.079031e-08
F03[O-O],-5.316396,1.058428e-07
SHED_DD,-5.285939,1.250616e-07
...,...,...
B05[P-Cl],0.000000,1.000000e+00
B05[P-F],0.000000,1.000000e+00
B05[P-P],0.000000,1.000000e+00
B05[F-I],0.000000,1.000000e+00


In [34]:
#Ranksum test <0.05
Ranksums_marked = Ranksums_results[Ranksums_results['Pvalues'] <0.05]
print(Ranksums_marked.shape)
Ranksums_marked.head()

(279, 2)


,Statistics,Pvalues
SAdon,-5.661572,1.499923e-08
P_VSA_ppp_D,-5.659880,1.514787e-08
MATS3e,-5.389154,7.079031e-08
F03[O-O],-5.316396,1.058428e-07
SHED_DD,-5.285939,1.250616e-07


In [35]:
X_ranksums_data=X_NAomit_data[Ranksums_marked.index]
X_ranksums_data.head()

,SAdon,P_VSA_ppp_D,MATS3e,F03[O-O],SHED_DD,B03[O-O],CATS2D_00_DD,CATS2D_04_DL,MATS1i,SHED_DA,...,MaxsCH3,SaaaC,C-030,SpPosA_L,O-059,WiA_Dz(e),T(N..O),Eta_epsi_2,N-072,CATS2D_08_LL
cid,,,,,,,,,,,,,,,,,,,,,
1401,54.032252,54.032252,-0.164768,0.0,3.000000,0.0,3.0,8.0,-0.311544,6.754792,...,2.184264,1.227741,0.0,2.105263,0.0,5.490414,165.0,0.834211,2.0,11.0
2375,60.694094,60.694094,-0.129887,1.0,1.000000,1.0,2.0,1.0,0.008452,5.669645,...,0.872742,0.000000,0.0,2.068966,0.0,4.422224,54.0,0.996223,1.0,4.0
3218,85.366686,85.366686,0.232993,2.0,1.000000,1.0,2.0,2.0,0.066951,2.871746,...,2.202759,0.000000,0.0,2.000000,0.0,4.875283,0.0,0.846032,0.0,5.0
3385,36.021501,36.021501,-0.107870,0.0,1.000000,0.0,2.0,0.0,-0.305789,1.754765,...,0.000000,0.000000,0.0,2.000000,0.0,1.358762,10.0,1.083069,2.0,0.0
4091,97.466341,97.466341,-0.422750,0.0,1.889882,0.0,3.0,0.0,-0.890241,1.889882,...,1.689259,0.000000,0.0,1.777778,0.0,1.605418,0.0,0.922222,0.0,0.0


In [36]:
#Scale data
Scaler = preprocessing.MinMaxScaler() #StandardScaler
Transformer =Scaler.fit(X_ranksums_data)
X_scaled_data=Transformer.transform(X_ranksums_data)
X_scaled_data =pd.DataFrame(X_scaled_data)
X_scaled_data.columns=X_ranksums_data.columns
X_scaled_data.index=Original_data.index
X_scaled_data.to_csv("./X_scaled_data.csv",sep=',',header=1,index=1)
joblib.dump(Transformer, './Scaler_transformer.pkl')
X_scaled_data.head()

,SAdon,P_VSA_ppp_D,MATS3e,F03[O-O],SHED_DD,B03[O-O],CATS2D_00_DD,CATS2D_04_DL,MATS1i,SHED_DA,...,MaxsCH3,SaaaC,C-030,SpPosA_L,O-059,WiA_Dz(e),T(N..O),Eta_epsi_2,N-072,CATS2D_08_LL
cid,,,,,,,,,,,,,,,,,,,,,
1401,0.085656,0.085656,0.393420,0.000000,0.189947,0.0,0.15,0.571429,0.513454,0.404071,...,0.860072,0.150667,0.0,0.506579,0.0,0.403211,0.033421,0.251650,0.250,0.255814
2375,0.096217,0.096217,0.446612,0.083333,0.063316,1.0,0.10,0.071429,0.797373,0.339157,...,0.343649,0.000000,0.0,0.450431,0.0,0.298966,0.010938,0.724111,0.125,0.093023
3218,0.135330,0.135330,1.000000,0.166667,0.063316,1.0,0.10,0.142857,0.849277,0.171787,...,0.867354,0.000000,0.0,0.343750,0.0,0.343180,0.000000,0.286123,0.000,0.116279
3385,0.057104,0.057104,0.480188,0.000000,0.063316,0.0,0.10,0.000000,0.518560,0.104970,...,0.000000,0.000000,0.0,0.343750,0.0,0.000000,0.002026,0.977370,0.250,0.000000
4091,0.154512,0.154512,0.000000,0.000000,0.119659,0.0,0.15,0.000000,0.000000,0.113052,...,0.665159,0.000000,0.0,0.000000,0.0,0.024071,0.000000,0.508309,0.000,0.000000


In [37]:
#Remove high correlation descriptors
X_cor_data=trimm_correlated(X_scaled_data,0.80)
X_cor_data.index=Original_data.index
X_cor_data.to_csv("./ML_data.csv",sep=',',header=1,index=1)
print_RxC(X_cor_data)
X_cor_data.head()

260 Moleculars × 87 Descriptors


,SAdon,MATS3e,F03[O-O],CATS2D_04_DL,MATS1i,SHED_DA,O-056,nRNHO,CATS2D_05_DL,CATS2D_00_DA,...,B08[N-O],GATS8v,CATS2D_07_LL,MATS7e,GATS1p,VE2sign_Dz(i),C-030,O-059,T(N..O),N-072
cid,,,,,,,,,,,,,,,,,,,,,
1401,0.085656,0.393420,0.000000,0.571429,0.513454,0.404071,0.000,0.0,0.272727,0.000,...,1.0,0.621819,0.200,0.351189,0.652889,0.737363,0.0,0.0,0.033421,0.250
2375,0.096217,0.446612,0.083333,0.071429,0.797373,0.339157,0.125,0.0,0.363636,0.125,...,1.0,0.581549,0.025,0.578201,0.475360,0.194267,0.0,0.0,0.010938,0.125
3218,0.135330,1.000000,0.166667,0.142857,0.849277,0.171787,0.000,0.0,0.181818,0.250,...,0.0,0.601115,0.150,0.419392,0.573471,0.949283,0.0,0.0,0.000000,0.000
3385,0.057104,0.480188,0.000000,0.000000,0.518560,0.104970,0.000,0.0,0.000000,0.000,...,0.0,0.000000,0.000,0.468814,0.450914,0.051565,0.0,0.0,0.002026,0.250
4091,0.154512,0.000000,0.000000,0.000000,0.000000,0.113052,0.000,0.0,0.000000,0.000,...,0.0,0.000000,0.000,1.000000,0.841813,0.166819,0.0,0.0,0.000000,0.000
